In [5]:
# MOunting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing the needed libraries

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('/content/train.csv')
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


In [8]:
# Drop the first column which is unnamed
df.drop(df.columns[0], axis=1, inplace=True)
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


In [9]:
df.isnull().sum()

,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36
Power,36


In [10]:
# Removing the cc in engine column and filling the missing values with the conditional median values
import re
import numpy as np


def cc_to_litres(engine):
    try:
        cc = int(str(engine).split()[0])
        return round(cc / 1000, 1)
    except:
        return None


def extract_engine_model(name):
    match = re.search(r'(\d\.\d)', str(name))
    return match.group(1) if match else None


df['Engine_Model'] = df['Engine'].apply(cc_to_litres)


# Extracting the engine info from name
df_temp = df.loc[df['Engine_Model'].isna(), 'Name'].apply(extract_engine_model)
df_temp = df_temp.astype(float)

df.loc[df['Engine_Model'].isna(), 'Engine_Model'] = df_temp

df['Engine_Model'] = df['Engine_Model'].astype(float)


df['Model'] = df['Name'].str.split().str[:2].str.join(' ')
df['Brand'] = df['Name'].str.split().str[0]


# Model-level median fill
df['Engine_Model'] = df.groupby('Model')['Engine_Model'].transform(
    lambda x: x.fillna(x.median())
)

# Brand-level median fill
df['Engine_Model'] = df.groupby('Brand')['Engine_Model'].transform(
    lambda x: x.fillna(x.median())
)

# Final fallback
df['Engine_Model'] = df['Engine_Model'].fillna(1.4)

df

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Engine_Model,Model,Brand
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50,1.6,Hyundai Creta,Hyundai
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,1.2,Honda Jazz,Honda
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00,1.2,Maruti Ertiga,Maruti
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74,2.0,Audi A4,Audi
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50,1.5,Nissan Micra,Nissan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.4 kmpl,1248 CC,74 bhp,5.0,7.88 Lakh,4.75,1.2,Maruti Swift,Maruti
5843,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.4 kmpl,1120 CC,71 bhp,5.0,NaN,4.00,1.1,Hyundai Xcent,Hyundai
5844,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.0 kmpl,2498 CC,112 bhp,8.0,NaN,2.90,2.5,Mahindra Xylo,Mahindra
5845,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.9 kmpl,998 CC,67.1 bhp,5.0,NaN,2.65,1.0,Maruti Wagon,Maruti


In [11]:
df.isnull().sum()

,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36
Power,36


In [12]:
df.drop('Engine', axis=1, inplace=True)
df.rename(columns={'Engine_Model': 'Engine'}, inplace=True)
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Power,Seats,New_Price,Price,Engine,Model,Brand
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,126.2 bhp,5.0,NaN,12.50,1.6,Hyundai Creta,Hyundai
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,88.7 bhp,5.0,8.61 Lakh,4.50,1.2,Honda Jazz,Honda
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,88.76 bhp,7.0,NaN,6.00,1.2,Maruti Ertiga,Maruti
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,140.8 bhp,5.0,NaN,17.74,2.0,Audi A4,Audi
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,63.1 bhp,5.0,NaN,3.50,1.5,Nissan Micra,Nissan


In [13]:
df.isnull().sum()

,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Power,36
Seats,38


In [14]:
# Power column filling missing values and transformation
import re

def clean_power(x):
    if pd.isna(x):
        return None
    match = re.search(r'([\d\.]+)', str(x))
    return float(match.group(1)) if match else None

df['Power'] = df['Power'].apply(clean_power)


# POWER
df['Power'] = df.groupby(
    ['Model', 'Engine', 'Fuel_Type']
)['Power'].transform(lambda x: x.fillna(x.median()))

df['Power'] = df.groupby(
    ['Model', 'Engine']
)['Power'].transform(lambda x: x.fillna(x.median()))

df['Power'] = df.groupby('Engine')['Power'].transform(
    lambda x: x.fillna(x.median())
)

df['Power'] = df.groupby('Brand')['Power'].transform(
    lambda x: x.fillna(x.median())
)

df['Power'] = df['Power'].fillna(75)


In [15]:
# SEATS
df['Seats'] = df.groupby('Model')['Seats'].transform(
    lambda x: x.fillna(x.median())
)

df['Seats'] = df.groupby('Brand')['Seats'].transform(
    lambda x: x.fillna(x.median())
)

df['Seats'] = df['Seats'].fillna(5)

df


,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Power,Seats,New_Price,Price,Engine,Model,Brand
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,126.20,5.0,NaN,12.50,1.6,Hyundai Creta,Hyundai
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,88.70,5.0,8.61 Lakh,4.50,1.2,Honda Jazz,Honda
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,88.76,7.0,NaN,6.00,1.2,Maruti Ertiga,Maruti
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,140.80,5.0,NaN,17.74,2.0,Audi A4,Audi
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,63.10,5.0,NaN,3.50,1.5,Nissan Micra,Nissan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.4 kmpl,74.00,5.0,7.88 Lakh,4.75,1.2,Maruti Swift,Maruti
5843,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.4 kmpl,71.00,5.0,NaN,4.00,1.1,Hyundai Xcent,Hyundai
5844,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.0 kmpl,112.00,8.0,NaN,2.90,2.5,Mahindra Xylo,Mahindra
5845,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.9 kmpl,67.10,5.0,NaN,2.65,1.0,Maruti Wagon,Maruti


In [16]:
df.isnull().sum()

,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Power,0
Seats,0


In [17]:
# see the rows only where df[mileage] is null
df.dropna(subset=['Mileage'], inplace=True)
df

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Power,Seats,New_Price,Price,Engine,Model,Brand
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,126.20,5.0,NaN,12.50,1.6,Hyundai Creta,Hyundai
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,88.70,5.0,8.61 Lakh,4.50,1.2,Honda Jazz,Honda
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,88.76,7.0,NaN,6.00,1.2,Maruti Ertiga,Maruti
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,140.80,5.0,NaN,17.74,2.0,Audi A4,Audi
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,63.10,5.0,NaN,3.50,1.5,Nissan Micra,Nissan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.4 kmpl,74.00,5.0,7.88 Lakh,4.75,1.2,Maruti Swift,Maruti
5843,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.4 kmpl,71.00,5.0,NaN,4.00,1.1,Hyundai Xcent,Hyundai
5844,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.0 kmpl,112.00,8.0,NaN,2.90,2.5,Mahindra Xylo,Mahindra
5845,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.9 kmpl,67.10,5.0,NaN,2.65,1.0,Maruti Wagon,Maruti


In [18]:
import re
import numpy as np

def extract_mileage_value(x):
    if pd.isna(x):
        return None
    match = re.search(r'([\d\.]+)', str(x))
    return float(match.group(1)) if match else None

df['Mileage_Num'] = df['Mileage'].apply(extract_mileage_value)

In [19]:
def detect_mileage_unit(x):
    x = str(x).lower()
    if 'kg' in x:     # km/kg → CNG
        return 'km/kg'
    return 'kmpl'     # petrol/diesel etc.

df['Mileage_Unit'] = df['Mileage'].apply(detect_mileage_unit)

In [20]:
df.loc[df['Mileage_Unit']=='kmpl', 'Mileage_Num'] = \
df.groupby(['Model','Engine','Fuel_Type'])['Mileage_Num'].transform(
    lambda x: x.fillna(x.median())
)

df.loc[df['Mileage_Unit']=='kmpl', 'Mileage_Num'] = \
df.groupby(['Model','Fuel_Type'])['Mileage_Num'].transform(
    lambda x: x.fillna(x.median())
)

df.loc[df['Mileage_Unit']=='kmpl', 'Mileage_Num'] = \
df.groupby('Fuel_Type')['Mileage_Num'].transform(
    lambda x: x.fillna(x.median())
)

df.loc[df['Mileage_Unit']=='km/kg', 'Mileage_Num'] = \
df.groupby(['Model','Fuel_Type'])['Mileage_Num'].transform(
    lambda x: x.fillna(x.median())
)

df.loc[df['Mileage_Unit']=='km/kg', 'Mileage_Num'] = \
df.groupby('Fuel_Type')['Mileage_Num'].transform(
    lambda x: x.fillna(x.median())
)

df.loc[df['Mileage_Unit']=='km/kg', 'Mileage_Num'] = \
df.loc[df['Mileage_Unit']=='km/kg', 'Mileage_Num'].fillna(
    df[df['Mileage_Unit']=='km/kg']['Mileage_Num'].median()
)

df['Mileage_Clean'] = df['Mileage_Num']
df.drop(['Mileage', 'Mileage_Num'], axis=1, inplace=True)

In [21]:
df

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Power,Seats,New_Price,Price,Engine,Model,Brand,Mileage_Unit,Mileage_Clean
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,126.20,5.0,NaN,12.50,1.6,Hyundai Creta,Hyundai,kmpl,19.67
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,88.70,5.0,8.61 Lakh,4.50,1.2,Honda Jazz,Honda,km/kg,13.00
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,88.76,7.0,NaN,6.00,1.2,Maruti Ertiga,Maruti,kmpl,20.77
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,140.80,5.0,NaN,17.74,2.0,Audi A4,Audi,kmpl,15.20
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,63.10,5.0,NaN,3.50,1.5,Nissan Micra,Nissan,kmpl,23.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,74.00,5.0,7.88 Lakh,4.75,1.2,Maruti Swift,Maruti,kmpl,28.40
5843,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,71.00,5.0,NaN,4.00,1.1,Hyundai Xcent,Hyundai,kmpl,24.40
5844,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,112.00,8.0,NaN,2.90,2.5,Mahindra Xylo,Mahindra,kmpl,14.00
5845,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,67.10,5.0,NaN,2.65,1.0,Maruti Wagon,Maruti,kmpl,18.90


In [22]:
df.isnull().sum()

,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Power,0
Seats,0
New_Price,5031


In [23]:
# Clean new price using the median value
import re
import numpy as np

# Clean new price
def clean_new_price(x):
    if pd.isna(x):
        return None

    x = str(x).lower().strip()
    match = re.search(r'([\d\.]+)', x)
    if not match:
        return None
    num = float(match.group(1))

    if "lakh" in x or "lac" in x:
        return num * 100000
    elif "cr" in x or "crore" in x:
        return num * 10000000
    else:
        return num

df['New_Price'] = df['New_Price'].apply(clean_new_price)

# Fill missing using hierarchy
df['New_Price'] = df.groupby(
    ['Model', 'Engine', 'Year']
)['New_Price'].transform(lambda x: x.fillna(x.median()))

df['New_Price'] = df.groupby(
    ['Model', 'Engine']
)['New_Price'].transform(lambda x: x.fillna(x.median()))

df['New_Price'] = df.groupby('Model')['New_Price'].transform(
    lambda x: x.fillna(x.median())
)

df['New_Price'] = df.groupby('Brand')['New_Price'].transform(
    lambda x: x.fillna(x.median())
)

df['New_Price'] = df['New_Price'].fillna(df['New_Price'].median())

df

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Power,Seats,New_Price,Price,Engine,Model,Brand,Mileage_Unit,Mileage_Clean
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,126.20,5.0,1621000.0,12.50,1.6,Hyundai Creta,Hyundai,kmpl,19.67
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,88.70,5.0,861000.0,4.50,1.2,Honda Jazz,Honda,km/kg,13.00
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,88.76,7.0,1144000.0,6.00,1.2,Maruti Ertiga,Maruti,kmpl,20.77
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,140.80,5.0,5314000.0,17.74,2.0,Audi A4,Audi,kmpl,15.20
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,63.10,5.0,948000.0,3.50,1.5,Nissan Micra,Nissan,kmpl,23.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,74.00,5.0,788000.0,4.75,1.2,Maruti Swift,Maruti,kmpl,28.40
5843,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,71.00,5.0,823000.0,4.00,1.1,Hyundai Xcent,Hyundai,kmpl,24.40
5844,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,112.00,8.0,1167000.0,2.90,2.5,Mahindra Xylo,Mahindra,kmpl,14.00
5845,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,67.10,5.0,529000.0,2.65,1.0,Maruti Wagon,Maruti,kmpl,18.90


In [24]:
df.isnull().sum()

,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Power,0
Seats,0
New_Price,0


In [25]:
df

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Power,Seats,New_Price,Price,Engine,Model,Brand,Mileage_Unit,Mileage_Clean
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,126.20,5.0,1621000.0,12.50,1.6,Hyundai Creta,Hyundai,kmpl,19.67
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,88.70,5.0,861000.0,4.50,1.2,Honda Jazz,Honda,km/kg,13.00
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,88.76,7.0,1144000.0,6.00,1.2,Maruti Ertiga,Maruti,kmpl,20.77
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,140.80,5.0,5314000.0,17.74,2.0,Audi A4,Audi,kmpl,15.20
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,63.10,5.0,948000.0,3.50,1.5,Nissan Micra,Nissan,kmpl,23.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,74.00,5.0,788000.0,4.75,1.2,Maruti Swift,Maruti,kmpl,28.40
5843,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,71.00,5.0,823000.0,4.00,1.1,Hyundai Xcent,Hyundai,kmpl,24.40
5844,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,112.00,8.0,1167000.0,2.90,2.5,Mahindra Xylo,Mahindra,kmpl,14.00
5845,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,67.10,5.0,529000.0,2.65,1.0,Maruti Wagon,Maruti,kmpl,18.90


In [26]:
# Do OneHOt encoding for column Fuel_type
df = pd.get_dummies(df, columns=['Fuel_Type'])
df = pd.get_dummies(df, columns=['Transmission'])

df

,Name,Location,Year,Kilometers_Driven,Owner_Type,Power,Seats,New_Price,Price,Engine,Model,Brand,Mileage_Unit,Mileage_Clean,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,126.20,5.0,1621000.0,12.50,1.6,Hyundai Creta,Hyundai,kmpl,19.67,True,False,False,True
1,Honda Jazz V,Chennai,2011,46000,First,88.70,5.0,861000.0,4.50,1.2,Honda Jazz,Honda,km/kg,13.00,False,True,False,True
2,Maruti Ertiga VDI,Chennai,2012,87000,First,88.76,7.0,1144000.0,6.00,1.2,Maruti Ertiga,Maruti,kmpl,20.77,True,False,False,True
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,140.80,5.0,5314000.0,17.74,2.0,Audi A4,Audi,kmpl,15.20,True,False,True,False
4,Nissan Micra Diesel XV,Jaipur,2013,86999,First,63.10,5.0,948000.0,3.50,1.5,Nissan Micra,Nissan,kmpl,23.08,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,Maruti Swift VDI,Delhi,2014,27365,First,74.00,5.0,788000.0,4.75,1.2,Maruti Swift,Maruti,kmpl,28.40,True,False,False,True
5843,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,First,71.00,5.0,823000.0,4.00,1.1,Hyundai Xcent,Hyundai,kmpl,24.40,True,False,False,True
5844,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Second,112.00,8.0,1167000.0,2.90,2.5,Mahindra Xylo,Mahindra,kmpl,14.00,True,False,False,True
5845,Maruti Wagon R VXI,Kolkata,2013,46000,First,67.10,5.0,529000.0,2.65,1.0,Maruti Wagon,Maruti,kmpl,18.90,False,True,False,True


In [27]:
from datetime import datetime

current_year = datetime.now().year
df['Car_Age'] = current_year - df['Year']
df['Car_Age'] = df['Car_Age'].clip(lower=0)
df

,Name,Location,Year,Kilometers_Driven,Owner_Type,Power,Seats,New_Price,Price,Engine,Model,Brand,Mileage_Unit,Mileage_Clean,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Car_Age
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,126.20,5.0,1621000.0,12.50,1.6,Hyundai Creta,Hyundai,kmpl,19.67,True,False,False,True,10
1,Honda Jazz V,Chennai,2011,46000,First,88.70,5.0,861000.0,4.50,1.2,Honda Jazz,Honda,km/kg,13.00,False,True,False,True,14
2,Maruti Ertiga VDI,Chennai,2012,87000,First,88.76,7.0,1144000.0,6.00,1.2,Maruti Ertiga,Maruti,kmpl,20.77,True,False,False,True,13
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,140.80,5.0,5314000.0,17.74,2.0,Audi A4,Audi,kmpl,15.20,True,False,True,False,12
4,Nissan Micra Diesel XV,Jaipur,2013,86999,First,63.10,5.0,948000.0,3.50,1.5,Nissan Micra,Nissan,kmpl,23.08,True,False,False,True,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,Maruti Swift VDI,Delhi,2014,27365,First,74.00,5.0,788000.0,4.75,1.2,Maruti Swift,Maruti,kmpl,28.40,True,False,False,True,11
5843,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,First,71.00,5.0,823000.0,4.00,1.1,Hyundai Xcent,Hyundai,kmpl,24.40,True,False,False,True,10
5844,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Second,112.00,8.0,1167000.0,2.90,2.5,Mahindra Xylo,Mahindra,kmpl,14.00,True,False,False,True,13
5845,Maruti Wagon R VXI,Kolkata,2013,46000,First,67.10,5.0,529000.0,2.65,1.0,Maruti Wagon,Maruti,kmpl,18.90,False,True,False,True,12


In [29]:
# Select the records
df_select = df[['Name', 'Year', 'Fuel_Type_Diesel','Fuel_Type_Petrol', 'Price']]
df_select

,Name,Year,Fuel_Type_Diesel,Fuel_Type_Petrol,Price
0,Hyundai Creta 1.6 CRDi SX Option,2015,True,False,12.50
1,Honda Jazz V,2011,False,True,4.50
2,Maruti Ertiga VDI,2012,True,False,6.00
3,Audi A4 New 2.0 TDI Multitronic,2013,True,False,17.74
4,Nissan Micra Diesel XV,2013,True,False,3.50
...,...,...,...,...,...
5842,Maruti Swift VDI,2014,True,False,4.75
5843,Hyundai Xcent 1.1 CRDi S,2015,True,False,4.00
5844,Mahindra Xylo D4 BSIV,2012,True,False,2.90
5845,Maruti Wagon R VXI,2013,False,True,2.65


In [30]:
# Filter the records
df_filter = df[df['Year'] > 2015]
df_filter

,Name,Location,Year,Kilometers_Driven,Owner_Type,Power,Seats,New_Price,Price,Engine,Model,Brand,Mileage_Unit,Mileage_Clean,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Car_Age
5,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,171.50,8.0,2100000.0,17.50,2.8,Toyota Innova,Toyota,kmpl,11.36,True,False,True,False,9
8,Maruti Ciaz Zeta,Kochi,2018,25692,First,103.25,5.0,1065000.0,9.95,1.5,Maruti Ciaz,Maruti,kmpl,21.56,False,True,False,True,7
14,Honda Amaze S i-Dtech,Kochi,2016,58950,First,98.60,5.0,854000.0,5.40,1.5,Honda Amaze,Honda,kmpl,25.80,True,False,False,True,9
15,Maruti Swift DDiS VDI,Jaipur,2017,25000,First,74.00,5.0,792000.0,5.99,1.2,Maruti Swift,Maruti,kmpl,28.40,True,False,False,True,8
26,Honda WRV i-VTEC VX,Kochi,2018,37430,First,88.70,5.0,1057000.0,9.90,1.2,Honda WRV,Honda,kmpl,17.50,False,True,False,True,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,Tata Hexa XTA,Jaipur,2016,39000,First,153.86,7.0,2100000.0,13.50,2.2,Tata Hexa,Tata,kmpl,17.60,True,False,True,False,9
5816,Tata Tiago 1.2 Revotron XT,Kochi,2017,15386,First,84.00,5.0,556000.0,5.11,1.2,Tata Tiago,Tata,kmpl,23.84,False,True,False,True,8
5825,Jaguar XF 2.2 Litre Luxury,Kochi,2016,31150,First,187.70,5.0,7264000.0,30.54,2.2,Jaguar XF,Jaguar,kmpl,16.36,True,False,True,False,9
5827,Tata Bolt Revotron XT,Chennai,2016,10000,First,88.70,5.0,777000.0,4.00,1.2,Tata Bolt,Tata,kmpl,17.57,False,True,False,True,9


In [31]:
# Rename the column name
df_rename = df.rename(columns={
    'Kilometers_Driven': 'KM_Driven',
    'New_Price': 'NewPrice',
    'Mileage_Clean': 'Mileage'
})

df = df_rename
df.head()

,Name,Location,Year,KM_Driven,Owner_Type,Power,Seats,NewPrice,Price,Engine,Model,Brand,Mileage_Unit,Mileage,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Car_Age
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,126.20,5.0,1621000.0,12.50,1.6,Hyundai Creta,Hyundai,kmpl,19.67,True,False,False,True,10
1,Honda Jazz V,Chennai,2011,46000,First,88.70,5.0,861000.0,4.50,1.2,Honda Jazz,Honda,km/kg,13.00,False,True,False,True,14
2,Maruti Ertiga VDI,Chennai,2012,87000,First,88.76,7.0,1144000.0,6.00,1.2,Maruti Ertiga,Maruti,kmpl,20.77,True,False,False,True,13
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,140.80,5.0,5314000.0,17.74,2.0,Audi A4,Audi,kmpl,15.20,True,False,True,False,12
4,Nissan Micra Diesel XV,Jaipur,2013,86999,First,63.10,5.0,948000.0,3.50,1.5,Nissan Micra,Nissan,kmpl,23.08,True,False,False,True,12


In [32]:
# Groupby method
df.groupby('Model').agg(
    Avg_Price=('Price', 'mean'),
    Avg_Mileage=('Mileage', 'mean'),
    Avg_Power=('Power', 'mean'),
    Count=('Name', 'count')
).reset_index()


,Model,Avg_Price,Avg_Mileage,Avg_Power,Count
0,Ambassador Classic,1.350000,12.800000,35.500000,1
1,Audi A3,19.287500,20.380000,143.000000,4
2,Audi A4,18.402208,16.477532,163.634416,77
3,Audi A6,22.058163,16.581837,186.290000,49
4,Audi A7,27.230000,9.600000,241.400000,3
...,...,...,...,...,...
202,Volvo S60,19.501429,16.108571,185.857143,7
203,Volvo S80,9.120000,14.550000,215.000000,2
204,Volvo V40,20.500000,20.050000,150.000000,4
205,Volvo XC60,18.480000,13.516667,200.666667,6


In [34]:
df.groupby('Fuel_Type_Diesel')['Mileage'].mean().reset_index(name='Avg_Mileage')

,Fuel_Type_Diesel,Avg_Mileage
0,False,17.576509
1,True,18.652661
